In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval
import matplotlib.patches as patches
from pydicom import dcmread, read_file
import skimage as sk
import cv2
import shutil
import os
from tqdm import tqdm

In [10]:
#extraer info de imagenes sin hallazgos para adjuntarlas al dataset
todo= pd.read_csv("/media/deadwolf/BEC4-9B89/chext14/Todas.csv")
bboxes=pd.read_csv("/media/deadwolf/BEC4-9B89/chext14/bboxes.csv")
data=pd.read_csv("/media/deadwolf/BEC4-9B89/chext14/Data.csv")

In [6]:
NoOpacities=data[data['Finding Labels']=='No Finding']

In [8]:
paths=NoOpacities['Image Index']
labels=NoOpacities['Finding Labels']

In [9]:
df = pd.DataFrame({'id': paths, 'label': labels})

In [11]:
df.to_csv('/media/deadwolf/BEC4-9B89/chext14/NoOp.csv', index=False)

In [10]:
info= pd.read_csv("bboxes.csv")
info.head()

,id,label,bboxes
0,00016568_026.png,Pneumothorax,"[[99.6910052910053,91.0222222222222,254.645502..."
1,00025252_032.png,Pneumothorax,"[[756.053333333333,681.851267361111,201.386666..."
2,00001946_029.png,Pneumothorax,"[[289.564444444444,396.269045138889,72.8177777..."
3,00017137_016.png,Pneumothorax,"[[170.097777777778,147.095711805556,287.857777..."
4,00014822_039.png,Pneumothorax,"[[644.551111111111,205.122378472222,194.56,623..."


In [9]:
#Determinemos la cantidad de elementos por etiqueta para distribuirlos equitativamente en los conjuntos de entrenamiento, prueba y validación
# train 0.6  validation 0.3 test 0.1
conteo = info['label'].value_counts()
conteo

label
Atelectasis     180
Effusion        153
Cardiomegaly    146
Infiltrate      123
No Finding      122
Pneumonia       120
Pneumothorax     98
Mass             85
Nodule           79
Name: count, dtype: int64

In [22]:
# reformateando imagenes del tipo (1024,1024,n) a (1024,1024)
root='/home/jair/COVID/NIHX14/images1024/'
for idd in info['id']:
    r=root+idd
    img=sk.io.imread(r)
    if img.shape == (1024,1024):
        pass
    else:
        imgr=img[...,0]
        sk.io.imsave('/home/jair/COVID/NIHX14/'+idd, imgr)

In [32]:
#mover a carpetas de train / test / validation (recordemos que una imagen puede presentar dos clases distintas)
rootd='/home/jair/COVID/NIHX14/images/1024/'
rooto='/home/jair/COVID/NIHX14/images1024/'
mode='test'
csv=pd.read_csv(mode+"_ord.csv")
for idd in tqdm(csv['id']):
    if os.path.exists(rootd+mode+'/'+idd):
        pass
    else:
        rd=rootd+mode
        ro=rooto+idd
        shutil.copy(ro, rd)

100%|███████████████████████████████████████| 109/109 [00:00<00:00, 1438.15it/s]
